In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
import asyncio
from langchain_deepseek import ChatDeepSeek


deepseek_client = ChatDeepSeek(
    api_key="deepseek-key",
    model="deepseek-chat",
    base_url="https://api.deepseek.com/v1",
)

async def main():
    async with MultiServerMCPClient(
        {
            "math": {
                "command": "python",
                # Replace with absolute path to your math_server.py file
                "args": ["server.py"],
                "transport": "stdio",
            },
            "weather": {
                # 使用URL形式的MCP服务器
                "url": "https://server.smithery.ai/@mcp-examples/weather/mcp?api_key=edd2f6f5af0c",
                "transport": "sse",  # MultiServerMCPClient只支持'stdio'和'sse'两种传输方式
            }
        }
    ) as client:
        agent = create_react_agent(
            deepseek_client,
            client.get_tools()
        )
        # 测试本地math服务
        math_response = await agent.ainvoke(
            {"messages": [{"role": "user", "content": "what's 3 + 5?"}]}
        )
        print("Math response:", math_response)
        
        # 测试远程weather服务
        weather_response = await agent.ainvoke(
            {"messages": [{"role": "user", "content": "what is the weather in Beijing?"}]}
        )
        print("Weather response:", weather_response)

if __name__ == "__main__":
    asyncio.run(main())